In [1]:
import os
import matplotlib.pyplot as plt
import netCDF4 as nc
from osgeo import gdal,osr
import numpy as np
from utils import getfiles

In [9]:
filePath = '/Users/bytedance/Downloads/MERRA2_400.tavgU_2d_slv_Nx.202306.nc4'
target = 'TS'
outtif = f'./out/{target}.tiff'
ncdata = nc.Dataset(filePath, mode='r')
ncdata.dimensions

{'lon': <class 'netCDF4._netCDF4.Dimension'>: name = 'lon', size = 576,
 'lat': <class 'netCDF4._netCDF4.Dimension'>: name = 'lat', size = 361,
 'time': <class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'time', size = 24}

In [7]:
time = ncdata.variables['time'][:]
time

/var/folders/b1/0fd1b6hs7lz0fm_mh346lybm0000gn/T/ipykernel_76654/615470046.py:1: RuntimeWarning: invalid value encountered in cast
  time = ncdata.variables['time'][:]
/var/folders/b1/0fd1b6hs7lz0fm_mh346lybm0000gn/T/ipykernel_76654/615470046.py:1: UserWarning: WARNING: valid_range not used since it
cannot be safely cast to variable data type
  time = ncdata.variables['time'][:]


masked_array(data=[0],
             mask=False,
       fill_value=999999,
            dtype=int32)

In [3]:
# longitude and latitude
lon = ncdata.variables['lon'][:]
lat = ncdata.variables['lat'][:]
# time = data.variables['time'][:]
#获取四个脚点坐标
LonMin,LatMax,LonMax,LatMin = [np.min(lon),np.max(lat),np.max(lon),np.min(lat)]
# 设置图像分辨率
row = len(lat)
col = len(lon)
Lon_Res=(LonMax - LonMin) / (float(col) - 1)
Lat_Res=(LatMax- LatMin) / (float(row) - 1)

GeoTransform = (LonMin,Lon_Res, 0, LatMax,0,-Lat_Res)
# 读取数据，处理异常值
var = ncdata.variables[target]
var_nans = var[:]
_FillValue = var._FillValue
var_nans[var_nans == _FillValue] = np.nan
tmp_data = var_nans[0,:,:]
# 灰度值转换
# tmp_data = 256 * (tmp_data - np.min(var))/(np.max(var)-np.min(var))
tmp_data = tmp_data[::-1]#这里是需要倒置一下的 这个很重要！！！！！

# 判断栅格数据的数据类型
if 'int8' in tmp_data.dtype.name:
    datatype = gdal.GDT_Byte
elif 'int16' in tmp_data.dtype.name:
    datatype = gdal.GDT_UInt16
else:
    datatype = gdal.GDT_Float32

driver = gdal.GetDriverByName('GTiff')
outRaster = driver.Create(outtif, col, row, 1, datatype)

outRaster.SetGeoTransform(tuple(GeoTransform))
# 获取数据集第一个波段，是从1开始，不是从0开始
outband = outRaster.GetRasterBand(1)
outband.WriteArray(ncdata)

# 数据投影信息
outRasterSRS = osr.SpatialReference()
# 代码4326表示WGS84坐标
outRasterSRS.ImportFromEPSG(4326)
outRaster.SetProjection(outRasterSRS.ExportToWkt())

outband.FlushCache()
out_ds=outband= None # 关闭spei_ds指针，注意必须关闭

KeyError: 'TS'